In [4]:

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600

Chapter 17 of `cite`{zaki2020data}: [Clustering Validation](https://www.cs.rpi.edu/~zaki/DMML/slides/pdf/ychap17.pdf)

# Clustering Validation Metrics

## Introduction to Clustering Validation

Clustering validation helps assess the quality of clustering results. There are three main types of validation measures:

1. **External measures**: Use external information like ground truth labels
2. **Internal measures**: Use only the data and clustering results
3. **Relative measures**: Compare different clusterings

We'll focus on several popular external and internal measures.

## External Validation Measures

### Contingency Table Basics

First, let's define some key quantities used in external measures:

- **TP (True Positives)**: Pairs in same cluster and same true partition
- **FP (False Positives)**: Pairs in same cluster but different partitions
- **TN (True Negatives)**: Pairs in different clusters and different partitions
- **FN (False Negatives)**: Pairs in different clusters but same partition

These can be computed from the contingency table:

In [1]:
def compute_contingency_table(cluster_labels, true_labels):
    """Compute contingency table: n_ij = number of items in cluster i and class j"""
    unique_clusters = np.unique(cluster_labels)
    unique_classes = np.unique(true_labels)
    n_ij = np.zeros((len(unique_clusters), len(unique_classes)))
    
    for cluster_idx, cluster in enumerate(unique_clusters):
        for class_idx, class_ in enumerate(unique_classes):
            n_ij[cluster_idx, class_idx] = np.sum((cluster_labels == cluster) & (true_labels == class_))
    
    return n_ij

All of the external measures rely on the $r \times k$  **contingency table** $\mathbf{N}$ that is induced by a clustering $\mathbf{C}$ and the ground-truth partitioning $\mathbf{T}$, defined as follows:  

$$
\mathbf{N}(i,j) = n_{ij}  = \left| C_i \cap T_{j} \right|
$$

The count $n_{ij}$ denotes the number of points that are common to cluster $C_i$ and ground-truth partition $T_{j}$.  

Let $n_{i} = |C_i|$ denote the number of points in cluster $C_i$, and let $m_{j} = |T_{j}|$ denote the number of points in partition  
$T_{j}$.  
The contingency table can be computed from $\mathbf{T}$ and $\mathbf{C}$ in $O(n)$ time by examining the partition and cluster labels, $y_i$ and  $\hat{y}_i$, for each point $\mathbf{x}_i \in \mathbf{D}$ and incrementing the corresponding count $n_{y_i\hat{y}_i}$.  

Let $G$ be a bipartite graph over the vertex set $V = \mathbf{C} \cup \mathbf{T}$, and let the edge set be $E = \{ (C_i, T_{j}) \}$ with  
edge weights $w(C_i,T_{j}) = n_{ij}$.  
A **matching** $M$ in $G$ is a subset of $E$, such that the edges in $M$ are pairwise nonadjacent, meaning they do not have a common vertex.  

### **Maximum Weight Matching**  
The maximum weight matching in $G$ is given as:

$$
\mathit{match} = \arg \max_M \left\{ \frac{w(M)}{n} \right\}
$$

where $w(M)$ is the sum of all edge weights in matching $M$,  given as $w(M) = \sum_{e \in M} w(e)$.  

### **Precision**  
Given cluster $C_i$, let $j_i$ denote the partition that contains the maximum number of points from $C_i$,
that is, $j_i = \max_{j=1}^k \{ n_{ij} \}$. 
The precision of a cluster $C_i$ is the same as its purity:

$$
\mathit{prec}_i = \frac{1}{n_{i}}\max_{j=1}^k \left\{ n_{ij} \right\} =
\frac{n_{ij_i}}{n_i}
$$

### **Recall**  
The recall of cluster $C_i$ is defined as:

$$
\mathit{recall}_i = \frac{n_{ij_i}}{|\mathbf{T}_{j_i}|} =\frac{n_{ij_i}}{m_{j_i}}
$$

where $m_{j_i} = |\mathbf{T}_{j_i}|$.  

### **F-Measure**  
The F-measure is the harmonic mean of the precision and recall values for  
each $C_i$:

$$
F_i = \frac{2}{\frac{1}{\mathit{prec}_i} + \frac{1}{\mathit{recall}_i}} =
\frac{2 \cdot \mathit{prec}_i \cdot \mathit{recall}_i}{\mathit{prec}_i + \mathit{recall}_i} =
\frac{2 \; n_{ij_i}}{n_{i} + m_{j_i}}
$$

The **F-measure** for the clustering $\mathbf{C}$ is the mean of clusterwise  
F-measure values:

$$
F = \frac{1}{r} \sum_{i=1}^r F_i
$$


### 1. Purity

Purity measures how "pure" each cluster is by the fraction of points from the majority class:

$$
\text{purity} = \frac{1}{n}\sum_{i=1}^r \max_{j=1}^k \{n_{ij}\}
$$

Where:
- $ n $ = total number of points
- $ r $ = number of clusters
- $ k $ = number of true classes
- $ n_{ij} $ = number of points in cluster $ i $ from class $ j $

In [2]:
def purity_score(cluster_labels, true_labels):
    contingency_matrix = compute_contingency_table(cluster_labels, true_labels)
    return np.sum(np.amax(contingency_matrix, axis=1)) / np.sum(contingency_matrix)

### 2. Maximum Matching

Finds the best matching between clusters and true classes to maximize overlap:

$$
\text{match} = \frac{1}{n}\max_M \sum_{(i,j)\in M} n_{ij}
$$

Where $ M $ is a matching between clusters and classes.

### Maximum Matching Example (Bipartite Graph Interpretation)

**Scenario**: Imagine we have clustered 10 fruits into 3 clusters, and we know their true categories (ground truth):

```
Cluster Assignments (C₁, C₂, C₃):
C₁: [apple1, apple2, banana1]
C₂: [orange1, orange2, apple3] 
C₃: [banana2, banana3, orange3, apple4]

True Categories (T₁=apples, T₂=oranges, T₃=bananas):
T₁: apple1, apple2, apple3, apple4
T₂: orange1, orange2, orange3
T₃: banana1, banana2, banana3
```

**Step 1: Build the Bipartite Graph**

We create edges between clusters and categories with weights equal to their overlap:

```
Clusters       Categories
   C₁ ------------- T₁ (weight=2: apple1,apple2)
      \------------ T₃ (weight=1: banana1)
   C₂ ------------- T₁ (weight=1: apple3)
      \------------ T₂ (weight=2: orange1,orange2)
   C₃ ------------- T₂ (weight=1: orange3)
      \------------ T₃ (weight=2: banana2,banana3)
      \------------ T₁ (weight=1: apple4)
```

**Step 2: Find Maximum Weight Matching**

We want to match each cluster to at most one category (and vice versa) to maximize total weight:

Possible matching options:
1. Match C₁-T₁ (2), C₂-T₂ (2), C₃-T₃ (2) → Total weight = 6
2. Match C₁-T₃ (1), C₂-T₂ (2), C₃-T₁ (1) → Total weight = 4

The first option gives us the maximum weight matching.

**Step 3: Calculate Maximum Matching Score**

$$
\text{match} = \frac{\text{Total matched weight}}{n} = \frac{2+2+2}{10} = 0.6
$$

In [8]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# Define contingency table
contingency = np.array([
    [2, 0, 1],  # C₁: 2 apples, 0 oranges, 1 banana
    [1, 2, 0],  # C₂: 1 apple, 2 oranges, 0 bananas
    [1, 1, 2]   # C₃: 1 apple, 1 orange, 2 bananas
])

# Find optimal matching
row_ind, col_ind = linear_sum_assignment(contingency, maximize=True)

# Perform indexing correctly
max_weight = contingency[row_ind, col_ind].sum()

print(f"Optimal matching: {list(zip(row_ind, col_ind))}")
print(f"Max weight: {max_weight}")

# Optimal matching: [(0, 0), (1, 1), (2, 2)]  # C₁-T₁, C₂-T₂, C₃-T₃

Optimal matching: [(0, 0), (1, 1), (2, 2)]
Max weight: 6


**Key Insights**:
1. This ensures one cluster maps to only one true category
2. Unlike purity, we don't double-count categories
3. The score ranges from 0 (worst) to 1 (perfect matching)
4. In our fruit example, 0.6 indicates moderate alignment between clusters and true categories

**Real-world analogy**: Imagine this as assigning each project team (cluster) to exactly one department (category) where you want to maximize the number of correctly assigned team members.

### 3. Precision, Recall and F-measure

For each cluster $ C_i $:

- **Precision**: Fraction of points in cluster from majority class
$$
\text{prec}_i = \frac{n_{ij_i}}{n_i}
$$

- **Recall**: Fraction of points from class found in cluster
$$
\text{recall}_i = \frac{n_{ij_i}}{m_{j_i}}
$$

- **F-measure**: Harmonic mean of precision and recall
$$
F_i = \frac{2 \cdot \text{prec}_i \cdot \text{recall}_i}{\text{prec}_i + \text{recall}_i} = \frac{2n_{ij_i}}{n_i + m_{j_i}}
$$

Overall F-measure is the average of cluster F-measures.


# Clustering Validation Metrics Tutorial

### Example 17.1: Good vs Bad Clustering on Iris Data

Let's examine two different clusterings of the Iris dataset using principal components:



#### Good Clustering Results:

![](img/Zaki-Chap17-Good-Clustering.png)

### Contingency Table

|   | iris-setosa | iris-versicolor | iris-virginica | Total (n_i) |
|---|------------|----------------|---------------|-------------|
| C₁ (squares)  | 0  | 47  | 14  | 61  |
| C₂ (circles)  | 50 | 0   | 0   | 50  |
| C₃ (triangles) | 0  | 3   | 36  | 39  |
| **Total (m_j)** | 50 | 50  | 50  | **n = 100** |

**Purity:** 0.887  
**Match:** 0.887  
**F:** 0.885  

#### Bad Clustering Results:

![](img/Zaki-Chap17-Bad-Clustering.png)

### Contingency Table

|   | iris-setosa | iris-versicolor | iris-virginica | Total (n_i) |
|---|------------|----------------|---------------|-------------|
| C₁ (squares)  | 30  | 0   | 0   | 30  |
| C₂ (circles)  | 20  | 4   | 0   | 24  |
| C₃ (triangles) | 0  | 46  | 50  | 96  |
| **Total (m_j)** | 50  | 50  | 50  | **n = 150** |

**Purity:** 0.667  
**Match:** 0.560  
**F:** 0.658  



### 4. Pairwise Measures

#### Jaccard Coefficient
$$
\text{Jaccard} = \frac{TP}{TP + FN + FP}
$$

#### Rand Index
$$
\text{Rand} = \frac{TP + TN}{TP + FN + FP + TN} = \frac{TP + TN}{\binom{n}{2}}
$$

## Internal Validation Measures

### Silhouette Coefficient

Measures how similar a point is to its own cluster compared to other clusters:

For each point $ x_i $:
$$
s_i = \frac{b_i - a_i}{\max(a_i, b_i)}
$$

Where:
- $ a_i $ = average distance to other points in same cluster
- $ b_i $ = min average distance to points in another cluster

Overall score is average of all $ s_i $.

## Python Example on Iris Dataset

In [10]:
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import (pairwise_distances, 
                            adjusted_rand_score,
                            silhouette_score)

# Load Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Cluster with K-means
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(X)

# External measures
contingency = compute_contingency_table(cluster_labels, y)
print(f"Purity: {purity_score(cluster_labels, y):.3f}")
print(f"Rand Index: {adjusted_rand_score(y, cluster_labels):.3f}")

# Internal measure
print(f"Silhouette Score: {silhouette_score(X, cluster_labels):.3f}")

# Pairwise measures
def pairwise_metrics(y_true, y_pred):
    tp = fp = fn = tn = 0
    n = len(y_true)
    
    for i in range(n):
        for j in range(i+1, n):
            same_cluster = y_pred[i] == y_pred[j]
            same_class = y_true[i] == y_true[j]
            
            if same_cluster and same_class:
                tp += 1
            elif same_cluster and not same_class:
                fp += 1
            elif not same_cluster and same_class:
                fn += 1
            else:
                tn += 1
                
    jaccard = tp / (tp + fn + fp)
    rand = (tp + tn) / (tp + fp + fn + tn)
    
    return jaccard, rand

jaccard, rand = pairwise_metrics(y, cluster_labels)
print(f"Jaccard: {jaccard:.3f}")
print(f"Rand Index: {rand:.3f}")

Purity: 0.887
Rand Index: 0.716
Silhouette Score: 0.551
Jaccard: 0.682
Rand Index: 0.874



## Interpretation of Results

- **Purity**: Higher is better (max 1.0)
- **Rand Index**: Higher is better (max 1.0)
- **Silhouette**: Ranges from -1 to 1, higher is better
- **Jaccard**: Higher is better (max 1.0)

## Choosing the Right Metric

1. Use external measures when ground truth is available
2. Use internal measures when no labels exist
3. Consider multiple metrics together for robust evaluation

Remember that no single metric tells the whole story - different metrics may be more appropriate for different clustering goals.


## Further Reading

Here are several real-world examples of bipartite matching problems that help explain the concept of maximum matching in clustering validation, along with relevant links for further reading:

### 1. **Stable Marriage Problem**
**Scenario**: Matching medical students to hospital residency programs based on mutual preferences    
**Visualization**:
```
Students   Hospitals
  A ──────── X (1st choice)
  B ──┬───── Y 
     └───── Z (tie)
  C ──────── Y (only option)
```
**Key Insight**: The Gale-Shapley algorithm finds stable pairings where no unmatched pair prefers each other over their current matches  
**Resource**: [Wikipedia - Stable Marriage Problem](https://en.wikipedia.org/wiki/Stable_marriage_problem)

---

### 2. **Roommate Assignment**
**Scenario**: Matching college students as roommates based on compatibility scores    
**Example Matrix**:
```
       Clean | Social | Studious
Alice    5   |   3    |    2
Bob      2   |   4    |    1
Carol    4   |   1    |    5
```
**Optimal Matching**: Alice-Clean (5), Bob-Social (4), Carol-Studious (5) → Total=14  
**Resource**: [Roommate Assignment Algorithms](https://en.wikipedia.org/wiki/Stable_roommates_problem)

---

### 3. **Thesis Advisor Matching**
**Scenario**: Assigning graduate students to faculty advisors based on research interests  
**Connection**: Similar to ensuring each cluster (advisor group) best represents one true category (research area)  
**Process Flow**:
1. Students rank advisors
2. Advisors rank students
3. Solve using maximum weight matching  
**Visualization**:
```
Students     Research Areas
  Sara ────┐
           ├── ML (Prof. Smith)
  John ────┘
  Emma ────── NLP (Prof. Lee)
```
**Resource**: 
- [Match Day 101: How does the medical residency match work?](https://scopeblog.stanford.edu/2024/03/13/match-day-medical-school-residency/)
- [Dr. Amin Saberi](https://stanford.edu/~saberi)

---

### 4. **Image Feature Matching**
**Scenario**: Matching keypoints between two images of the same object  
**Example**:
```
Image A Features   Image B Features
   Corner X ─────────── Edge P (distance=1.2)
   Blob Y ──┬────────── Corner Q (distance=0.8)
           └────────── Blob R (distance=1.5)
```
**Optimal Matching**: X-P (1.2), Y-Q (0.8) → Total=2.0  
**Algorithm**: Hungarian algorithm on distance matrix  
**Resource**: [OpenCV Feature Matching](https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html)

---

### 5. **Job Applicant Matching**
**Scenario**: Assigning applicants to job openings based on skill fit    
**Matching Table**:
```
Applicants  Jobs       Fit Score
  Amy ───── Data Sci ─── 0.9
  Sam ──┬── Engineer ── 0.7
        └── Analyst ── 0.8
  Zoe ───── Engineer ── 0.6
```
**Maximum Matching**: Amy-Data Sci (0.9), Sam-Analyst (0.8), Zoe-Engineer (0.6) → Total=2.3  
**Resource**: [LinkedIn Matching Algorithm](https://engineering.linkedin.com/blog/2016/06/how-linkedin-uses-machine-learning-to-match-job-seekers-with-jo)

---

### Comparative Summary Table

| Application          | Cluster Validation Analogy          | Key Metric               | Common Algorithm        |
|----------------------|-------------------------------------|--------------------------|-------------------------|
| Stable Marriage      | Cluster-to-class 1:1 mapping        | Preference satisfaction  | Gale-Shapley            |
| Roommate Assignment  | Intra-cluster homogeneity           | Compatibility score      | Hungarian Algorithm     |
| Advisor Matching     | Aligning clusters with true labels  | Research overlap         | Deferred Acceptance     |
| Image Matching       | Cross-cluster correspondence        | Feature distance         | RANSAC                  |
| Job Applicant        | Purity maximization                 | Skill fit percentage     | Linear Sum Assignment   |

These examples demonstrate how maximum matching principles apply across domains while maintaining the core idea of optimal constrained assignments - exactly what we do when evaluating clustering quality against ground truth.